# Core

> Conatains methods for transferring.

In [ ]:
# default_exp core

In [ ]:
#hide
from nbdev.showdoc import *
from fastai.tabular.all import *
from transfertab.utils import *
import json

In [ ]:
# This will contain the functions for transferring embeddings

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
df1 = pd.read_csv(path/'adult.csv')
splits1 = RandomSplitter(valid_pct=0.2)(range_of(df1))
to1 = TabularPandas(df1, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits1)
dls1 = to1.dataloaders(bs=64)
learn1 = tabular_learner(dls1, metrics=accuracy)

In [ ]:
learn1.model.embeds

ModuleList(
  (0): Embedding(10, 6)
  (1): Embedding(17, 8)
  (2): Embedding(8, 5)
  (3): Embedding(16, 8)
  (4): Embedding(7, 5)
  (5): Embedding(6, 4)
  (6): Embedding(3, 3)
  (7): Embedding(3, 3)
)

Aim is to take these embeddings and transfer them to other models

In [ ]:
learn1.model.embeds[0].weight, learn1.model.embeds[0].weight.shape

(Parameter containing:
 tensor([[-5.6853e-03,  1.8688e-03,  1.4476e-02,  9.7890e-03,  1.4115e-02,
           5.7125e-03],
         [-9.1271e-03, -1.5946e-03,  5.6058e-03,  3.0356e-03,  2.5559e-03,
          -1.2680e-02],
         [-2.1623e-03, -4.1510e-05, -1.0180e-02,  1.3648e-02,  1.5476e-02,
          -1.8098e-03],
         [ 5.6572e-03,  1.5083e-02,  7.9600e-03, -7.3476e-04, -1.1631e-02,
          -7.4402e-03],
         [ 3.3497e-03, -3.0329e-03,  2.3689e-03, -3.3820e-03,  1.5509e-02,
          -9.7973e-03],
         [ 1.2162e-02,  1.0208e-02,  1.4835e-03,  8.3532e-03,  1.1189e-02,
           9.2965e-04],
         [ 8.5163e-03, -1.2888e-02,  8.2518e-03,  3.5525e-03,  4.3668e-03,
           3.4388e-03],
         [ 6.6009e-03, -5.5191e-03,  7.2820e-03,  1.3600e-02, -2.4659e-03,
          -6.2848e-04],
         [ 1.0715e-03, -3.1847e-03, -6.2840e-03, -6.0915e-03,  7.6644e-04,
           4.6805e-03],
         [ 1.5399e-02, -8.3873e-03, -8.9908e-03,  3.5097e-04, -7.0670e-03,
           

In [ ]:
learn1.fit(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.371636,0.351842,0.838298,00:04


In [ ]:
#hide
meta = {
    "categories":['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex'],
    "workclass": {
        "classes": ['nan', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov',
           ' Federal-gov', ' Local-gov', ' ?', ' Without-pay',
           ' Never-worked'],
    },
    'education': {
        "classes": ['nan', ' Assoc-acdm', ' Masters', ' HS-grad', ' Prof-school', ' 7th-8th',
       ' Some-college', ' 11th', ' Bachelors', ' Assoc-voc', ' 10th',
       ' 9th', ' Doctorate', ' 12th', ' 1st-4th', ' 5th-6th',
       ' Preschool']
    },
    "marital-status": {
        "classes": ['nan', ' Married-civ-spouse', ' Divorced', ' Never-married', ' Widowed',
       ' Married-spouse-absent', ' Separated', ' Married-AF-spouse']
    },
    "occupation": {
        "classes": ["nan", ' Exec-managerial', ' Prof-specialty', ' Other-service',
       ' Handlers-cleaners', ' Craft-repair', ' Adm-clerical', ' Sales',
       ' Machine-op-inspct', ' Transport-moving', ' ?',
       ' Farming-fishing', ' Tech-support', ' Protective-serv',
       ' Priv-house-serv', ' Armed-Forces']
    },
    "relationship": {
        "classes": ['nan', ' Wife', ' Not-in-family', ' Unmarried', ' Husband', ' Own-child',
       ' Other-relative']
    },
    "race": {
        "classes": ['nan', ' White', ' Black', ' Asian-Pac-Islander', ' Amer-Indian-Eskimo',
       ' Other']
    },
    "sex": {
        "classes": ["Female", "Male", "nan"]
    }
}

In [ ]:
#hide
extractembeds(learn1.model, "embeds", meta, "../data/adults.json")
print("done")

done


In [ ]:
#We add robot to our "race" column
new_rows = pd.DataFrame([[49,'Private',101320,'Assoc-acdm',12.0,'Married-civ-spouse','Exec-managerial','Wife','Robot','Female',0,1902,40,'United-States','>=50k'],
                        [18,'Private',182308,'Bachelors',10.0,'Never-married','?','Own-child','Other','Male',0,0,23,'United-States','<50k']],
                        columns=df1.columns)
df2 = df1.copy()
df2 = df2.append(new_rows, ignore_index=True)
df2.tail()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
32558,53,Private,157069,Assoc-acdm,12.0,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>=50k
32559,32,Local-gov,217296,HS-grad,9.0,Married-civ-spouse,Transport-moving,Wife,White,Female,4064,0,22,United-States,<50k
32560,26,Private,182308,Some-college,10.0,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,40,United-States,<50k
32561,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,Exec-managerial,Wife,Robot,Female,0,1902,40,United-States,>=50k
32562,18,Private,182308,Bachelors,10.0,Never-married,?,Own-child,Other,Male,0,0,23,United-States,<50k


In [ ]:
splits2 = RandomSplitter(valid_pct=0.2)(range_of(df2))
to2 = TabularPandas(df2, procs=[Categorify, FillMissing,Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits2)
dls2 = to2.dataloaders(bs=64)
learn2 = tabular_learner(dls2, metrics=accuracy)

In [ ]:
learn2.model.embeds

ModuleList(
  (0): Embedding(11, 6)
  (1): Embedding(19, 8)
  (2): Embedding(10, 6)
  (3): Embedding(18, 8)
  (4): Embedding(9, 5)
  (5): Embedding(8, 5)
  (6): Embedding(5, 4)
  (7): Embedding(3, 3)
)

In [ ]:
# export
class TabTransfer:
    def __init__(self, new_learner):
        self.new_cat_names = new_learner.dls.cat_names
        self.new_all_classes = new_learner.dls.classes
        self.new_learner = new_learner
        
    def init_from_json(self, path):
        '''
        `path` can either a json file or a directory containing multiple json files.
        The json should contain 
            a `categories` key with all the categories in that json, 
            a key for each category in `categories` inside which there should be
                a `classes` key with all classes of that key
                an `embeddings` key with the embeddings for that category
        '''
        path = Path(path)
        json_file = {}
        if path.is_file(): 
            f = open(path, 'rb')
            json_file = json.load(f)
        elif path.is_dir():
            json_file = {"categories": []}
            for json_f in path.glob("*.json"):
                f = open(json_f, 'rb')
                file = json.load(f)
                json_file["categories"] += file["categories"]
                del file["categories"]
                json_file.update(file)
        self.old_cat_names = json_file['categories']
        classes_dict = dict.fromkeys(self.old_cat_names)
        embed_dict = dict.fromkeys(self.old_cat_names)
        
        for i in self.old_cat_names:
            classes_dict[i] = json_file[i]['classes']
            embed_dict[i] = json_file[i]['embeddings']
        self.old_all_classes = classes_dict # contains the cat variable with all its classes
        self.old_all_embeds = embed_dict # contains the cat variable with all its embeddings
        
    def mapping(self, cat_names_to_transfer):
        mapping_dict = dict.fromkeys(cat_names_to_transfer)
        for curr_cat in cat_names_to_transfer:
            if (curr_cat in self.old_cat_names and curr_cat in self.new_cat_names):
                mapping_dict[curr_cat] = curr_cat
            else:
                print(self.new_cat_names)
                mapping_cat = input(f"\nWhat do you want to map {curr_cat} to : \r")
                mapping_dict[curr_cat] = mapping_cat
        return mapping_dict
                
        
    def transfer(self, cat_names_to_transfer, layergroup, mapping_dict, verbose=False):
        for curr_cat in cat_names_to_transfer:
            if not (curr_cat in self.old_cat_names and curr_cat in self.new_cat_names):
                continue
            old_cat_idx = self.old_cat_names.index(curr_cat)
            new_cat_idx = self.new_cat_names.index(curr_cat)
            
            # TODO: Make it so that this isn't required by taking care of this.
            try: assert (len(tabobj.old_all_embeds[curr_cat][0]) == getattr(self.new_learner.model, layergroup)[new_cat_idx].embedding_dim)
            except: 
                print(f"Encountered an error for variable {curr_cat}: Make sure embeddings dimensions are same for {self.old_all_embeds[curr_cat]} with size {len(tabobj.old_all_embeds[curr_cat])}, and {self.new_learner.model.embeds[new_cat_idx]} with size {self.new_learner.model.embeds[new_cat_idx].embedding_dim}")
                print("Moving on to other cat vars")
                continue
            
            old_curr_classes = self.old_all_classes[curr_cat]
            new_curr_classes = self.new_all_classes[curr_cat]
            
            torch_embeds = torch.FloatTensor(self.old_all_embeds[curr_cat])
            weights_mean = torch_embeds.mean(0)
            
            if verbose: print(f'mean is {weights_mean} for {torch_embeds}')

            # Case where some class in old, but not in new isn't being handled rn.

            for new_curr_class in new_curr_classes:
                new_curr_class_idx = new_curr_classes.o2i[new_curr_class]
                if verbose: print(f"{new_curr_class_idx}, {type(new_curr_class_idx)}")
                
                if new_curr_class in old_curr_classes:
                    old_curr_class_idx = old_curr_classes.index(new_curr_class)
                    if verbose: print(f'Transferring weights for class {new_curr_class}, cat {curr_cat} from previous weights')
                    if verbose: print(f"old weight for class is {self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :]}")
                    tempwgt1 = self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :]
                    tempwgt2 = torch_embeds[old_curr_class_idx, :]
                    
                    self.new_learner.model.embeds[new_cat_idx].weight.data[new_curr_class_idx, :] = torch_embeds[old_curr_class_idx, :].detach().clone()
                    self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :].required_grad = True
                    if verbose: print(f"new weight for class is {self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :]}")
                else:
                    if verbose: print(f'Transferring weights for class {new_curr_class}, cat {curr_cat} using mean')
                    if verbose: print(f"old weight for class is {self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :]}")
                    self.new_learner.model.embeds[new_cat_idx].weight.data[new_curr_class_idx, :] = weights_mean
                    self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :].required_grad = True
                    if verbose: print(f"new weight for class is {self.new_learner.model.embeds[new_cat_idx].weight[new_curr_class_idx, :]}")

In [ ]:
tabobj = TabTransfer(learn1)

In [ ]:
tabobj.init_from_json("../data/adults.json")

In [ ]:
tabobj.old_cat_names, tabobj.old_all_classes

(['workclass',
  'education',
  'marital-status',
  'occupation',
  'relationship',
  'race',
  'sex'],
 {'workclass': ['nan',
   ' Private',
   ' Self-emp-inc',
   ' Self-emp-not-inc',
   ' State-gov',
   ' Federal-gov',
   ' Local-gov',
   ' ?',
   ' Without-pay',
   ' Never-worked'],
  'education': ['nan',
   ' Assoc-acdm',
   ' Masters',
   ' HS-grad',
   ' Prof-school',
   ' 7th-8th',
   ' Some-college',
   ' 11th',
   ' Bachelors',
   ' Assoc-voc',
   ' 10th',
   ' 9th',
   ' Doctorate',
   ' 12th',
   ' 1st-4th',
   ' 5th-6th',
   ' Preschool'],
  'marital-status': ['nan',
   ' Married-civ-spouse',
   ' Divorced',
   ' Never-married',
   ' Widowed',
   ' Married-spouse-absent',
   ' Separated',
   ' Married-AF-spouse'],
  'occupation': ['nan',
   ' Exec-managerial',
   ' Prof-specialty',
   ' Other-service',
   ' Handlers-cleaners',
   ' Craft-repair',
   ' Adm-clerical',
   ' Sales',
   ' Machine-op-inspct',
   ' Transport-moving',
   ' ?',
   ' Farming-fishing',
   ' Tech-supp

In [ ]:
mapping_dict = tabobj.mapping(["race", "workclass", "gender"])

['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'education-num_na']

What do you want to map gender to : 
sex


In [ ]:
tabobj.transfer(["race", "workclass", "gender"], "embeds", mapping_dict, verbose = True)

mean is tensor([ 0.0026,  0.0024, -0.0051, -0.0007]) for tensor([[ 0.0006, -0.0192, -0.0002,  0.0069],
        [ 0.0045,  0.0120, -0.0178, -0.0027],
        [-0.0107,  0.0151,  0.0019,  0.0083],
        [ 0.0127,  0.0003, -0.0127, -0.0057],
        [ 0.0130,  0.0087, -0.0119, -0.0109],
        [-0.0043, -0.0026,  0.0104,  0.0002]])
0, <class 'int'>
Transferring weights for class #na#, cat race using mean
old weight for class is tensor([ 0.0026,  0.0024, -0.0051, -0.0007], grad_fn=<SliceBackward>)
new weight for class is tensor([ 0.0026,  0.0024, -0.0051, -0.0007], grad_fn=<SliceBackward>)
1, <class 'int'>
Transferring weights for class  Amer-Indian-Eskimo, cat race from previous weights
old weight for class is tensor([ 0.0130,  0.0087, -0.0119, -0.0109], grad_fn=<SliceBackward>)
new weight for class is tensor([ 0.0130,  0.0087, -0.0119, -0.0109], grad_fn=<SliceBackward>)
2, <class 'int'>
Transferring weights for class  Asian-Pac-Islander, cat race from previous weights
old weight for c